In [466]:
import pandas as pd
from pandas import json_normalize
import json
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Get Data

In [13]:
test_df = pd.read_csv('/Users/maksimtrebusinin/Downloads/test_csv.csv')
test_df

In [269]:
data_list = []
columns = test_df.columns
with open('/Users/maksimtrebusinin/Downloads/fingerprints_with_os_checked.json') as file:
    for line in file:
        try:
            # Разбираем каждую строку как отдельный JSON-объект
            obj = json.loads(line.strip())
            data_list.append(obj)
        except json.JSONDecodeError as e:
            print(f"Ошибка при декодировании строки: {e}")

print(f'Count of JSON objects: {len(data_list)}')

keylist = [el.split('.') for el in columns]

def get_data(key, data):
    if len(key) == 1:
        key = key[0]
        try:
            return data[key]
        except Exception as e:
            return np.nan
    elif len(key) == 2:
        key1, key2, = key
        try:
            return data[key1][key2]
        except Exception as e:
            return np.nan
    elif len(key) == 3:
        key1, key2, key3 = key
        try:
            return data[key1][key2][key3]
        except Exception as e:
            return np.nan

data_frame = pd.DataFrame(columns=columns)
count = 1
for data in data_list:
    obj = []
    for key in keylist:
        obj.append(get_data(key=key,data=data))
    prom_df = pd.DataFrame([obj], columns=columns)
    data_frame = pd.concat([data_frame, prom_df], ignore_index=True)
    if count % 200 == 0:
        print(f'Lengh of data: {len(data_frame)}')
data_frame.info()

In [200]:
with open('/Users/maksimtrebusinin/Downloads/scamalytics_rechecked.json') as f:
    rechecked_data = json.load(f)
rech_keys = [x for x in rechecked_data['79.210.243.3'].keys()]
objects = []
for el in rechecked_data:
    objects.append(np.array([x for x in rechecked_data[el].values()]))
rech_df = pd.DataFrame(data = objects, columns = rech_keys)

In [204]:
rech_df.info()

# EDA

In [241]:
def is_there_dupl(key):
    key_columns = [x for x in data_frame.columns if key in x]
    if data_frame[key_columns].duplicated().sum() > 0:
        key_columns.append('ip')
        print(data_frame[key_columns][data_frame[key_columns].duplicated() == True])
    else:
        print('No duplicates!')

In [233]:
tls_columns = [x for x in data_frame.columns if 'tls' in x]
if data_frame[tls_columns].drop(['tls.extensions','tls.ciphers'], axis=1).duplicated().sum() > 0:
    tls_columns.append('ip')
    data_frame[tls_columns]
else:
    print('No duplicates!')

In [270]:
is_there_dupl('tcp')

In [271]:
data_frame.drop([869, 871, 872, 6490, 7363], axis=0, inplace=True)
data_frame.drop('http2.sent_frames', axis=1, inplace=True)

In [272]:
# Разделение akamai_fingerprint на составляющие
data_frame['akamai_fingerprint.settings'] = data_frame['http2.akamai_fingerprint'].apply(lambda x: x.split('|')[0])
data_frame['akamai_fingerprint.window_update'] = data_frame['http2.akamai_fingerprint'].apply(lambda x: x.split('|')[1])
data_frame['akamai_fingerprint.priority_weights'] = data_frame['http2.akamai_fingerprint'].apply(lambda x: x.split('|')[2])
data_frame['akamai_fingerprint.flags'] = data_frame['http2.akamai_fingerprint'].apply(lambda x: x.split('|')[3])

# Удаление исходного столбца
data_frame.drop(columns=['http2.akamai_fingerprint'], inplace=True)

In [273]:
js_columns = ['js_fingerprint.fingerprint','js_fingerprint.fingerprints.persistentFp','js_fingerprint.fingerprints.uniqueFp',
             'js_fingerprint.profile.deviceMemory','js_fingerprint.profile.screenResolution','js_fingerprint.profile.userAgentData',
             'js_fingerprint.profile.platform']
data_frame.drop([x for x in data_frame.columns if x not in 'js' in x and x not in js_columns], axis=1, inplace=True)

In [321]:
data_frame.reset_index(drop=True, inplace=True)

In [329]:
data_frame.drop(['tcpip.cap_length','tcpip.dst_port','tcpip.src_port','tcpip.ip.id',
                'tcpip.ip.tos','tcpip.ip.dst_ip','tcpip.ip.src_ip','tcpip.tcp.checksum',
                'tcpip.tcp.options','tcpip.tcp.options_order','tcpip.tcp.seq', 'tcpip.tcp.options'], axis=1, inplace=True)

In [351]:
data_frame['tls_version'] = data_frame['tls.ja3'].apply(lambda x: x.split(',')[0])
data_frame['ciphers'] = data_frame['tls.ja3'].apply(lambda x: x.split(',')[1])
data_frame['expansions'] = data_frame['tls.ja3'].apply(lambda x: x.split(',')[2])
data_frame['supp_gr'] = data_frame['tls.ja3'].apply(lambda x: x.split(',')[3])
data_frame['signature_algorithms'] = data_frame['tls.ja3'].apply(lambda x: x.split(',')[4])

data_frame.drop(['tls.ciphers','tls.extensions','tls.tls_version_record','tls.peetprint',
                'tls.peetprint','tls.client_random','tls.session_id'], axis=1, inplace=True)

In [369]:
data_frame.drop(['os_prediction.platform_mismatch','blacklist_info.blacklist_info.ip','blacklist_info.blacklist_info.FoundIn'],
               axis=1, inplace = True)

In [373]:
data_frame.info()

In [380]:
data_frame_with_out_js = data_frame.drop([x for x in data_frame.columns if 'js' in x], axis=1)
del js

In [388]:
for col in data_frame_with_out_js.columns:
    print(f'{col} - {data_frame_with_out_js[col][0]}')

In [394]:
data_frame_with_out_js.dropna(inplace=True)
columns_for_switch_type = ['tls.tls_version_negotiated','tcpip.ip.ttl','tcpip.ip.ip_version','tcpip.tcp.mss',
                          'tcpip.tcp.window','blacklist_info.blacklist_info.TotalBlacklists','blacklist_info.blacklist_info.CountFoundIn',
                          'akamai_fingerprint.window_update','tls_version']
for col in columns_for_switch_type:
    data_frame_with_out_js[col] = data_frame_with_out_js[col].astype('int')

In [397]:
for col in data_frame_with_out_js.select_dtypes('object').columns:
    data_frame_with_out_js[col] = LabelEncoder().fit_transform(data_frame_with_out_js[col])
data_frame_with_out_js['os_prediction.highest'].value_counts()

In [ ]:
# Android: 0
# Linux: 1
# Mac OS: 2
# Windows: 3
# iOS: 4

In [403]:
data_frame_with_out_js[data_frame_with_out_js['os_prediction.highest'] == 2]

In [404]:
for col in data_frame_with_out_js.columns:
    print(f'{col} - {data_frame_with_out_js[col].nunique()} unique values')

In [410]:
plt.figure(figsize=(20,20))
sns.heatmap(data_frame_with_out_js.select_dtypes('int').corr(), annot = True, cmap='coolwarm', vmin=-1, center=0, vmax=1, fmt='.2f')
plt.show()

# Training and Testing models

In [420]:
models = [XGBClassifier(), CatBoostClassifier(verbose=False), RandomForestClassifier()]
X,y = data_frame_with_out_js.drop('os_prediction.highest', axis=1), data_frame_with_out_js['os_prediction.highest']
X_tr, X_t, y_tr, y_t = train_test_split(X,y, test_size=0.25, random_state=42, stratify=y)
for model in models:
    model.fit(X_tr,y_tr)
    pred = model.predict(X_t)
    print(f'Accuarcy: {accuracy_score(y_t,pred)}')
    print(f'F1_score: {f1_score(y_t,pred, average="weighted")}')
    print('----------')

In [425]:
xgb = XGBClassifier()
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
for scoring in ['f1_macro', 'f1_micro', 'f1_weighted','recall_weighted']:
    cross_val = cross_val_score(xgb, X,y, cv=cv, scoring=scoring)
    print(f'{scoring} - {cross_val.mean()}')

In [433]:
def cross_val(df, list_of_scores, model):
    X,y = df.drop('os_prediction.highest', axis=1), df['os_prediction.highest']
    cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
    for scoring in list_of_scores:
        cross_v = cross_val_score(model, X,y, cv=cv, scoring=scoring)
        print(f'{scoring} - {cross_v.mean()}')

In [434]:
test = data_frame_with_out_js.drop(['user_agent','tls.ja3_hash','tls.peetprint_hash'], axis=1)
xgb = XGBClassifier()
cross_val(test, ['f1_macro', 'f1_micro', 'f1_weighted','recall_weighted'], xgb)

In [451]:
test.drop_duplicates(inplace=True)

In [452]:
X,y = test.drop('os_prediction.highest', axis=1), test['os_prediction.highest']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y, random_state=42)
print(f'y_train labesl: {np.bincount(y_train)}')
print(f'y_test labesl: {np.bincount(y_test)}')

rnf = RandomForestClassifier()
rnf.fit(X_train,y_train)
y_pred = rnf.predict(X_test)
print(f'f1_macro: {f1_score(y_test, y_pred, average="macro")}')

In [ ]:
import joblib

joblib.dump(rnf, 'random_forest_model.pkl')

# Interpretation

In [453]:
importances = rnf.feature_importances_
feature_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)
plt.figure(figsize=(10, 6))
plt.barh(feature_df['Feature'], feature_df['Importance'], color='skyblue')
plt.xlabel('Важность признака')
plt.ylabel('Признак')
plt.title('Важность признаков в модели Random Forest')
plt.gca().invert_yaxis()  # Инвертируем ось Y для удобства чтения
plt.show()

In [460]:
grouped = test.groupby('os_prediction.highest')

In [467]:
# Построение KDE для 'tcpip.tcp.window'
plt.figure(figsize=(10, 6))
for os, group in grouped:
    sns.kdeplot(group['tcpip.tcp.window'], label=os, shade=True)
plt.title('KDE для tcpip.tcp.window')
plt.xlabel('Значение tcpip.tcp.window')
plt.ylabel('Плотность')
plt.legend(title='Операционная система')
plt.show()

# Построение KDE для 'tcpip.ip.ttl'
plt.figure(figsize=(10, 6))
for os, group in grouped:
    sns.kdeplot(group['tcpip.ip.ttl'], label=os, shade=True)
plt.title('KDE для tcpip.ip.ttl')
plt.xlabel('Значение tcpip.ip.ttl')
plt.ylabel('Плотность')
plt.legend(title='Операционная система')
plt.show()